In [13]:
%%writefile main.cpp
#include <iostream>
#include <cmath>
#include <iomanip>
#include <vector>
#include <string>
#include <fstream> // Required for file input
#include <sstream> // Required for parsing lines from file

// Function to calculate f(v) = 2*c1*v - 2*c2*v^(-3)
// This function represents the derivative of (Power/Velocity) with respect to v, set to 0 for optimal range.
double f_drone(double v, double c1, double c2) {
    // Ensure v is positive to avoid division by zero or issues with negative v in pow(v, -3)
    if (v <= 1e-9) { // Using a small threshold instead of direct zero comparison
        // std::cerr << "Warning: Kecepatan v (" << v << ") sangat kecil atau nol dalam f_drone(). Mengembalikan nilai besar." << std::endl;
        return 1e12; // Return a very large number to indicate an issue / push away from this region
    }
    return 2.0 * c1 * v - (2.0 * c2 / (v * v * v));
}

// Function to calculate f'(v) = 2*c1 + 6*c2*v^(-4)
// This is the derivative of f_drone(v) with respect to v.
double df_drone(double v, double c1, double c2) {
    // Ensure v is positive
    if (v <= 1e-9) {
        // std::cerr << "Warning: Kecepatan v (" << v << ") sangat kecil atau nol dalam df_drone(). Mengembalikan nilai besar." << std::endl;
        return 1e12; // Return a very large number
    }
    return 2.0 * c1 + (6.0 * c2 / (v * v * v * v));
}

// Structure to hold results for each test case
struct TestCaseResult {
    double c1_param; // Renamed to avoid conflict with local c1 in some contexts
    double c2_param; // Renamed
    double v0_initial;
    double v_optimal_numerical;
    int iterations_taken;
    double v_optimal_analytical;
    double relative_error_percent;
    std::string status_message;
};

// Newton-Raphson implementation for finding optimal drone speed
// Returns a TestCaseResult structure
TestCaseResult newton_raphson_solver(double c1, double c2, double v0, double tolerance, int max_iter, bool print_iterations_detail = false) {
    double v_current = v0;
    int iter = 0;
    TestCaseResult result = {c1, c2, v0, v0, 0, 0.0, -1.0, "Max Iterations Reached"}; // Default status

    if (v0 <= 0) {
        result.status_message = "Invalid Initial Guess (v0 <= 0)";
        result.iterations_taken = 0;
        result.v_optimal_numerical = v0; // Or some indicator like NaN
        return result;
    }
     if (c1 <= 0 || c2 <= 0) {
        result.status_message = "Invalid Parameters (c1 or c2 <= 0)";
        result.iterations_taken = 0;
        result.v_optimal_numerical = v0;
        return result;
    }


    if (print_iterations_detail) {
        std::cout << "\n--- Iteration Details for c1=" << c1 << ", c2=" << c2 << ", v0=" << v0 << " ---\n";
        std::cout << std::setw(10) << "Iteration"
                  << std::setw(18) << "v_i (m/s)"
                  << std::setw(18) << "f(v_i)"
                  << std::setw(18) << "f'(v_i)"
                  << std::setw(18) << "v_{i+1} (m/s)"
                  << std::setw(20) << "|v_{i+1}-v_i|" << std::endl;
        std::cout << std::string(102, '-') << std::endl;
    }

    while (iter < max_iter) {
        double f_val = f_drone(v_current, c1, c2);
        double df_val = df_drone(v_current, c1, c2);

        if (std::abs(df_val) < 1e-10) { // Derivative is too close to zero
            if (print_iterations_detail) std::cerr << "Error: Derivative too close to zero at v = " << v_current << std::endl;
            result.status_message = "Derivative Zero";
            result.v_optimal_numerical = v_current;
            result.iterations_taken = iter;
            return result;
        }

        double v_next = v_current - f_val / df_val;
        double change = std::abs(v_next - v_current);

        if (v_next <= 0) { // Solution is heading to a non-physical region
             if (print_iterations_detail) std::cerr << "Error: Next velocity is non-positive (v_next = " << v_next << ")" << std::endl;
            result.status_message = "Non-Positive Velocity";
            result.v_optimal_numerical = v_current; // Keep the last valid velocity
            result.iterations_taken = iter;
            return result;
        }


        if (print_iterations_detail) {
            std::cout << std::fixed << std::setprecision(8)
                      << std::setw(10) << iter
                      << std::setw(18) << v_current
                      << std::setw(18) << f_val
                      << std::setw(18) << df_val
                      << std::setw(18) << v_next
                      << std::setw(20) << change << std::endl;
        }

        if (change < tolerance) {
            if (print_iterations_detail) std::cout << "\nConverged after " << iter + 1 << " iterations!" << std::endl;
            result.status_message = "Converged";
            result.v_optimal_numerical = v_next;
            result.iterations_taken = iter + 1;
            return result;
        }

        v_current = v_next;
        iter++;
    }

    // If loop finishes, max iterations were reached without meeting tolerance
    if (print_iterations_detail) std::cerr << "Warning: Maximum iterations (" << max_iter << ") reached without convergence." << std::endl;
    result.v_optimal_numerical = v_current; // Store the last calculated velocity
    result.iterations_taken = iter;
    return result;
}

void print_summary_table(const std::vector<TestCaseResult>& results) {
    std::cout << "\n\n--- Summary Table of Newton-Raphson Results for Drone Optimization ---" << std::endl;
    std::cout << std::setw(10) << "c1"
              << std::setw(10) << "c2"
              << std::setw(15) << "v0_initial"
              << std::setw(20) << "v_optimal_num"
              << std::setw(12) << "Iterations"
              << std::setw(20) << "v_optimal_analyt"
              << std::setw(18) << "Rel.Error (%)"
              << std::setw(25) << "Status" << std::endl;
    std::cout << std::string(130, '-') << std::endl;

    for (const auto& res : results) {
        std::cout << std::fixed << std::setprecision(5)
                  << std::setw(10) << res.c1_param
                  << std::setw(10) << res.c2_param
                  << std::setw(15) << res.v0_initial
                  << std::setw(20) << res.v_optimal_numerical
                  << std::setw(12) << res.iterations_taken
                  << std::setw(20) << res.v_optimal_analytical;
        if (res.relative_error_percent >= 0) {
            std::cout << std::setw(18) << std::setprecision(5) << res.relative_error_percent;
        } else {
            std::cout << std::setw(18) << "N/A"; // For cases where relative error isn't applicable
        }
        std::cout << std::setw(25) << res.status_message << std::endl;
    }
    std::cout << std::string(130, '-') << std::endl;
}


int main() {
    std::vector<TestCaseResult> all_results;
    std::string input_filename = "synthetic_data.txt";
    std::ifstream infile(input_filename);

    if (!infile.is_open()) {
        std::cerr << "Error: Tidak dapat membuka file input: " << input_filename << std::endl;
        std::cerr << "Pastikan file 'synthetic_data.txt' ada di direktori yang sama atau jalankan generator data terlebih dahulu." << std::endl;
        return 1;
    }

    std::cout << "Membaca data dari " << input_filename << " untuk Optimasi Kecepatan Drone...\n" << std::endl;

    std::string line;
    int line_number = 0;
    // Header for detailed iteration output (optional, can be controlled)
    // bool show_detailed_iterations_for_first_case = true;

    while (std::getline(infile, line)) {
        line_number++;
        std::istringstream iss(line);
        double c1_in, c2_in, v0_in, tol_in;
        int max_iter_in;

        if (!(iss >> c1_in >> c2_in >> v0_in >> tol_in >> max_iter_in)) {
            std::cerr << "Error parsing baris ke-" << line_number << ": " << line << std::endl;
            std::cerr << "Format yang diharapkan: c1 c2 v0_awal epsilon iterasi_maks" << std::endl;
            continue; // Skip this line and try the next
        }

        // Optional: print details for one specific case or first few cases for debugging
        // bool print_details_this_case = (line_number == 1 && show_detailed_iterations_for_first_case);
        bool print_details_this_case = false; // Set to true to see iterations for a case

        TestCaseResult current_res = newton_raphson_solver(c1_in, c2_in, v0_in, tol_in, max_iter_in, print_details_this_case);

        // Calculate analytical solution and relative error
        if (c1_in > 0 && c2_in > 0) {
            current_res.v_optimal_analytical = std::pow(c2_in / c1_in, 0.25);
            if (current_res.status_message == "Converged" && current_res.v_optimal_analytical > 0) {
                current_res.relative_error_percent = std::abs(current_res.v_optimal_numerical - current_res.v_optimal_analytical) / current_res.v_optimal_analytical * 100.0;
            } else {
                current_res.relative_error_percent = -1.0; // Indicates N/A or error in convergence/analytical
            }
        } else {
            current_res.v_optimal_analytical = -1.0; // N/A if c1 or c2 are not positive
            current_res.relative_error_percent = -1.0;
        }
        all_results.push_back(current_res);
    }

    infile.close();

    if (all_results.empty() && line_number == 0) {
        std::cout << "File " << input_filename << " kosong atau tidak ada data yang valid." << std::endl;
    } else if (all_results.empty() && line_number > 0) {
        std::cout << "Tidak ada data yang berhasil diproses dari " << input_filename << "." << std::endl;
    }
    else {
        print_summary_table(all_results);
    }

    return 0;
}



Overwriting main.cpp


In [9]:
%%writefile data.cpp
#include <iostream>
#include <fstream>  // Untuk operasi file
#include <vector>
#include <random>   // Untuk menghasilkan angka acak
#include <iomanip>  // Untuk std::fixed dan std::setprecision

// Struktur untuk menyimpan satu set parameter input
struct InputParameters {
    double c1;
    double c2;
    double v0_initial_guess;
    double epsilon;
    int max_iterations;
};

// Fungsi untuk menghasilkan nilai double acak dalam rentang [min, max]
double random_double(double min_val, double max_val) {
    static std::mt19937 generator(std::random_device{}()); // Mesin Angka Acak Mersenne Twister
    std::uniform_real_distribution<double> distribution(min_val, max_val);
    return distribution(generator);
}

int main() {
    int num_datasets;

    std::cout << "----------------------------------------------------" << std::endl;
    std::cout << "  Generator Data Sintetis untuk Optimasi Kecepatan Drone" << std::endl;
    std::cout << "----------------------------------------------------" << std::endl;
    std::cout << "Masukkan jumlah set data yang ingin dibuat: ";
    std::cin >> num_datasets;

    if (num_datasets <= 0) {
        std::cerr << "Jumlah set data harus positif." << std::endl;
        return 1;
    }

    std::vector<InputParameters> datasets(num_datasets);

    // Tentukan rentang untuk parameter acak
    // Anda bisa menyesuaikan rentang ini sesuai kebutuhan
    double c1_min = 0.05, c1_max = 0.5;
    double c2_min = 50.0, c2_max = 500.0;
    double v0_min = 1.0, v0_max = 20.0; // Tebakan awal kecepatan
    double epsilon_val = 1e-5; // Toleransi error bisa dibuat tetap atau diacak
    int max_iter_val = 100;    // Iterasi maksimum bisa dibuat tetap atau diacak

    for (int i = 0; i < num_datasets; ++i) {
        datasets[i].c1 = random_double(c1_min, c1_max);
        datasets[i].c2 = random_double(c2_min, c2_max);

        // Untuk tebakan awal v0, kita bisa mencoba membuatnya sedikit lebih "pintar"
        // agar dekat dengan solusi analitik v_opt = (c2/c1)^(1/4)
        // Ini hanya untuk menghasilkan tebakan awal yang lebih baik, bukan solusi sebenarnya.
        double analytical_v_approx = pow(datasets[i].c2 / datasets[i].c1, 0.25);
        // Hasilkan v0 di sekitar perkiraan analitik, misal +/- 50%
        double v0_guess_min = std::max(v0_min, analytical_v_approx * 0.5);
        double v0_guess_max = std::min(v0_max, analytical_v_approx * 1.5);
        if (v0_guess_min > v0_guess_max) { // Jika rentang tidak valid karena c1/c2 ekstrim
            datasets[i].v0_initial_guess = random_double(v0_min, v0_max); // Fallback ke rentang umum
        } else {
            datasets[i].v0_initial_guess = random_double(v0_guess_min, v0_guess_max);
        }
        // Pastikan v0 tidak nol atau negatif
        if (datasets[i].v0_initial_guess <= 1e-6) {
            datasets[i].v0_initial_guess = 1.0; // Nilai default jika terlalu kecil
        }


        datasets[i].epsilon = epsilon_val; // Menggunakan nilai epsilon yang tetap
        datasets[i].max_iterations = max_iter_val; // Menggunakan iterasi maks yang tetap
    }

    // Tulis data ke file synthetic_data.txt
    std::ofstream outfile("synthetic_data.txt");
    if (!outfile.is_open()) {
        std::cerr << "Error: Tidak dapat membuka file synthetic_data.txt untuk ditulis." << std::endl;
        return 1;
    }

    // Atur presisi untuk output file
    outfile << std::fixed << std::setprecision(8);

    for (const auto& params : datasets) {
        outfile << params.c1 << " "
                << params.c2 << " "
                << params.v0_initial_guess << " "
                << params.epsilon << " "
                << params.max_iterations << std::endl;
    }

    outfile.close();

    std::cout << "\n" << num_datasets << " set data telah berhasil dibuat dan disimpan ke synthetic_data.txt" << std::endl;
    std::cout << "Format per baris: c1 c2 v0_awal epsilon iterasi_maks" << std::endl;

    return 0;
}


Writing data.cpp


In [14]:
!g++ main.cpp -o main
!g++ data.cpp -o generate_data -std=c++11

In [15]:
!./generate_data
!./main

----------------------------------------------------
  Generator Data Sintetis untuk Optimasi Kecepatan Drone
----------------------------------------------------
Masukkan jumlah set data yang ingin dibuat: 100

100 set data telah berhasil dibuat dan disimpan ke synthetic_data.txt
Format per baris: c1 c2 v0_awal epsilon iterasi_maks
Membaca data dari synthetic_data.txt untuk Optimasi Kecepatan Drone...



--- Summary Table of Newton-Raphson Results for Drone Optimization ---
        c1        c2     v0_initial       v_optimal_num  Iterations    v_optimal_analyt     Rel.Error (%)                   Status
----------------------------------------------------------------------------------------------------------------------------------
   0.42489 242.99032        3.31916             4.89022           6             4.89022           0.00000                Converged
   0.31645 491.86572        8.76309             6.27891           6             6.27891           0.00000                Conver